In [2]:
import pandas as pd
import numpy as np
import formulas
import psycopg2
from tqdm import tqdm
from sklearn.cluster import KMeans

In [3]:
viewName = 'vw_usuario_kpi'

In [4]:
conn = psycopg2.connect(
    host="ec2-52-86-115-245.compute-1.amazonaws.com",
    database="d92em23mmlh07d",
    user="yjixuohawbxqpj",
    password="61a11b1ebe328f19fc17be47b54382fc78a89a613d3eaea7633ff4cf4a3d73ad"
)

cur = conn.cursor()

In [5]:
cur.execute(
    "SELECT a.attname " +
    "FROM pg_attribute a JOIN pg_class t on a.attrelid = t.oid JOIN pg_namespace s on t.relnamespace = s.oid " +
    "WHERE a.attnum > 0 AND NOT a.attisdropped AND t.relname = '" + viewName + "' AND s.nspname = 'public' " +
    "ORDER BY a.attnum;"
)
columnsResult = cur.fetchall()

In [6]:
title = []
for i in columnsResult:
    title.append(i[0])

In [7]:
cur.execute("select * from " + viewName)
usuarioKPI = cur.fetchall()

In [8]:
df = pd.DataFrame(usuarioKPI)
for n,col in enumerate(title):
    df = df.rename(columns={n:col})

df = df[df.dt_referencia == max(df.dt_referencia)]

df = df.drop(['dt_referencia', 'nm_usuario', 'dt_registro','id_plano','vl_plano','qt_jogo_criado','qt_avaliacao','vl_nota','qt_atividade','qt_atividade_login','qt_atividade_cadastro','qt_atividade_jogo','qt_atividade_busca','dt_updated'], axis=1)

df.nm_plano = [  formulas.one_hot(df.nm_plano, _) for _ in df.nm_plano]

df.set_index('id_usuario', inplace=True)

df

,vl_idade,vl_dias_registrado,nm_plano,vl_dias_no_plano,qt_hist_jogo_criado,qt_hist_avaliacao,vl_hist_nota,qt_hist_atividade,qt_hist_atividade_login,qt_hist_atividade_cadastro,qt_hist_atividade_jogo,qt_hist_atividade_busca
id_usuario,,,,,,,,,,,,
1,0,256,[1.0],256,10,5,3.2627,121,121,0,0,0
2,26,247,[1.0],247,0,4,4.2118,0,0,0,0,0
3,41,232,[1.0],232,0,3,4.0000,0,0,0,0,0
4,77,213,[1.0],213,0,4,3.9686,0,0,0,0,0
5,57,211,[1.0],211,0,5,4.6258,0,0,0,0,0
6,47,191,[1.0],191,0,5,4.4513,0,0,0,0,0
7,21,175,[1.0],175,0,2,5.0000,0,0,0,0,0
8,40,153,[1.0],153,0,1,3.5000,0,0,0,0,0
9,72,145,[1.0],145,0,6,3.9235,0,0,0,0,0


In [9]:
kmeans = KMeans(n_clusters=5, random_state=0).fit(df.values)
kmeans.labels_

array([4, 2, 2, 2, 2, 3, 3, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32)

In [10]:
id = df.index
cluster = kmeans.labels_

result = pd.DataFrame()
result['id'] = id
result['group'] = cluster + 2
result.head(5)

,id,group
0,1,6
1,2,4
2,3,4
3,4,4
4,5,4


In [11]:
insertList = []

for i in range(len(result)):
    line = result.iloc[i]
    insert = "insert into tb_usuario_grupo values(nextval('sq_usuario_grupo'), "+ str(line.id) +", "+ str(line.group) +", 'sim', null, current_timestamp);"
    insertList.append(insert)

In [12]:
for i in tqdm(range(len(insertList))):
    l = insertList[i]
    cur.execute(l)

100%|██████████| 21/21 [00:04<00:00,  4.68it/s]


In [13]:
cur.execute("commit;")

In [14]:
cur.close
conn.close

<function connection.close>